In [ ]:
!pip install pyDOE

In [ ]:
import jax, flax, optax, time, pickle
import os
import jax.numpy as np
import numpy as onp
from functools import partial
from pyDOE import lhs
from typing import Sequence
import json
from tensorflow_probability.substrates import jax as tfp

In [ ]:
# Run on the first GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from jax.extend.backend import get_backend
print(get_backend().platform)

In [ ]:
# Hyperparameters
architecture_list = [[1, 1], [2, 1], [5, 1], [10, 1], [20, 1], [40, 1],
                     [5, 5, 1], [10, 10, 1], [20, 20, 1], [40, 40, 1],
                     [5, 5, 5, 1], [10, 10, 10, 1], [20, 20, 20, 1],
                     [40, 40, 40, 1]] # NN architecture list
lr = 1e-4 # learning rate
num_epochs = 15000 # number of training epochs